In [1]:
##Dream 11 winning predictions

In [2]:
##import libraries

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [5]:
data=pd.read_excel(r"C:\Users\GIS\Desktop\wps_download\Dream 11_Day 2_Dataset (matches).xlsx")

In [6]:
data.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [9]:
print(data.isnull().sum())

id                    0
season                0
city                 51
date                  0
match_type            0
player_of_match       5
venue                 0
team1                 0
team2                 0
toss_winner           0
toss_decision         0
winner                5
result                0
result_margin        19
target_runs           3
target_overs          3
super_over            0
method             1074
umpire1               0
umpire2               0
dtype: int64


In [10]:
data=data.dropna(subset=['winner'])

In [12]:
print(data.isnull().sum())

id                    0
season                0
city                 51
date                  0
match_type            0
player_of_match       0
venue                 0
team1                 0
team2                 0
toss_winner           0
toss_decision         0
winner                0
result                0
result_margin        14
target_runs           0
target_overs          0
super_over            0
method             1069
umpire1               0
umpire2               0
dtype: int64


In [13]:
data.shape

(1090, 20)

In [21]:
team_city_map={
'Mumbai Indians':'Mumbai',
'Chennnai Super Kings':"Chennai",'Kings XI Punjab':'Punjab',
       'Delhi Daredevils':'Delhi','Kolkata Knight Riders':'Kolkata',
       'Rajasthan Royals':'Rajasthan', 'Deccan Chargers':'Hyderabad','Kochi Tuskers Kerala':'Kochi', 'Pune Warriors':'Pune', 'Sunrisers Hyderabad':'Hyderabad',
       'Gujarat Lions':'Gujurat', 'Rising Pune Supergiants':'Pune',
       'Rising Pune Supergiant':'Pune', 'Delhi Capitals':'Delhi', 'Punjab Kings':'Punjab',
       'Lucknow Super Giants':'Lucknow', 'Gujarat Titans':'Gujurat',
       'Royal Challengers Bengaluru':'Bangalore'
}

In [23]:
data.team2.unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
       'Royal Challengers Bangalore', 'Deccan Chargers',
       'Kings XI Punjab', 'Delhi Daredevils', 'Mumbai Indians',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Gujarat Titans', 'Lucknow Super Giants',
       'Royal Challengers Bengaluru'], dtype=object)

In [24]:
data['team1_home_city']=data['team1'].map(team_city_map)

In [27]:
data['team2_home_city']=data.team2.map(team_city_map)

In [28]:
data.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,...,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2,team1_home_city,team2_home_city
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,...,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen,NaN,Kolkata
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,...,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri,Punjab,NaN
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,...,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar,Delhi,Rajasthan
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,...,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper,Mumbai,NaN
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,...,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan,Kolkata,Hyderabad


In [32]:
data['Home_advantage_team1']=(data['team1_home_city']==data.city).astype(int)

In [33]:
data['Home_advantage_team2']=(data['team2_home_city']==data.city).astype(int)

In [34]:
data.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,...,target_runs,target_overs,super_over,method,umpire1,umpire2,team1_home_city,team2_home_city,Home_advantage_team1,Home_advantage_team2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,...,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen,NaN,Kolkata,0,0
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,...,241.0,20.0,N,NaN,MR Benson,SL Shastri,Punjab,NaN,0,0
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,...,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar,Delhi,Rajasthan,1,0
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,...,166.0,20.0,N,NaN,SJ Davis,DJ Harper,Mumbai,NaN,1,0
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,...,111.0,20.0,N,NaN,BF Bowden,K Hariharan,Kolkata,Hyderabad,1,0


In [36]:
x=data[['city','venue','team1','team2','toss_decision','target_runs','target_overs','Home_advantage_team1','Home_advantage_team2']]

In [37]:
x.head()

,city,venue,team1,team2,toss_decision,target_runs,target_overs,Home_advantage_team1,Home_advantage_team2
0,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,field,223.0,20.0,0,0
1,Chandigarh,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,bat,241.0,20.0,0,0
2,Delhi,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,bat,130.0,20.0,1,0
3,Mumbai,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,bat,166.0,20.0,1,0
4,Kolkata,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,bat,111.0,20.0,1,0


In [38]:
y=data.winner

In [40]:
y.head()

0          Kolkata Knight Riders
1            Chennai Super Kings
2               Delhi Daredevils
3    Royal Challengers Bangalore
4          Kolkata Knight Riders
Name: winner, dtype: object

In [45]:
for col in x.select_dtypes(include='object').columns:
    x[col]=LabelEncoder().fit_transform(x[col].astype(str))

C:\Users\GIS\AppData\Local\Temp\ipykernel_7340\958827920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col]=LabelEncoder().fit_transform(x[col].astype(str))
C:\Users\GIS\AppData\Local\Temp\ipykernel_7340\958827920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col]=LabelEncoder().fit_transform(x[col].astype(str))
C:\Users\GIS\AppData\Local\Temp\ipykernel_7340\958827920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [46]:
y=LabelEncoder().fit_transform(y)

In [48]:
x.head()

,city,venue,team1,team2,toss_decision,target_runs,target_overs,Home_advantage_team1,Home_advantage_team2
0,2,23,16,8,1,223.0,20.0,0,0
1,7,40,6,0,0,241.0,20.0,0,0
2,10,16,3,13,0,130.0,20.0,1,0
3,26,55,10,16,0,166.0,20.0,1,0
4,23,14,8,1,0,111.0,20.0,1,0


In [50]:
y

array([ 8,  0,  3, ..., 13, 18,  8])

In [51]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=43)

In [52]:
x_train_full,x_val,y_train_full,y_val=train_test_split(x_train,y_train,test_size=0.2,stratify=y_train,random_state=43)

In [53]:
x_train.shape

(872, 9)

In [54]:
print(x_train.shape,x_test.shape,x.shape)

(872, 9) (218, 9) (1090, 9)


In [55]:
model=XGBClassifier()

In [57]:
model.fit(x_train_full,y_train_full,
          eval_set=[[x_val,y_val]],verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [58]:
train_accuracy=accuracy_score(y_train_full,model.predict(x_train_full))
train_accuracy

0.9971305595408895

In [60]:
test_acc=accuracy_score(y_test,model.predict(x_test))
test_acc

0.555045871559633

In [61]:
from sklearn.model_selection import GridSearchCV

In [88]:
param_grids={
         'max_depth':[3,5],
    'learning_rate':[0.05,0.01,0.1],
    'n_estimators':[100,125,75],
    'subsample':[0.7,0.9,1.0],
    'colsample_bytree':[0.7,0.8,1.0]
        }

In [65]:
grid_search=GridSearchCV(estimator=model,param_grid=param_grids,cv=3,n_jobs=1,scoring='accuracy',verbose=2)

In [66]:
grid_search.fit(x_train_full,y_train_full)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=0.7; total time=   1.4s
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=0.7; total time=   1.3s
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=0.7; total time=   1.1s
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=0.9; total time=   1.2s
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=0.9; total time=   1.3s
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=0.9; total time=   1.3s
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=1.0; total time=   1.1s
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=1.0; total time=   1.3s
[CV] END 

[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=125, subsample=1.0; total time=   1.6s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=75, subsample=0.7; total time=   1.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=75, subsample=0.7; total time=   1.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=75, subsample=0.7; total time=   1.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=75, subsample=0.9; total time=   1.0s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=75, subsample=0.9; total time=   0.9s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=75, subsample=0.9; total time=   1.2s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=75, subsample=1.0; total time=   1.2s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=75

[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1.0; total time=   1.7s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=125, subsample=0.7; total time=   1.9s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=125, subsample=0.7; total time=   2.1s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=125, subsample=0.7; total time=   2.1s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=125, subsample=0.9; total time=   2.2s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=125, subsample=0.9; total time=   1.9s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=125, subsample=0.9; total time=   2.1s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=125, subsample=1.0; total time=   2.0s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=5, n_estimators=125,

[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=5, n_estimators=75, subsample=1.0; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimat

[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=125, subsample=1.0; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=125, subsample=1.0; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=75, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=75, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=75, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=75, subsample=0.9; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=75, subsample=0.9; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=75, subsample=0.9; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=75, subsam

[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estimators=100, subsample=1.0; total time=   0.8s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estimators=100, subsample=1.0; total time=   0.8s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estimators=125, subsample=0.7; total time=   0.9s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estimators=125, subsample=0.7; total time=   0.9s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estimators=125, subsample=0.7; total time=   0.9s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estimators=125, subsample=0.9; total time=   1.0s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estimators=125, subsample=0.9; total time=   1.1s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estimators=125, subsample=0.9; total time=   1.0s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=5, n_estima

[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=75, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=75, subsample=1.0; total time=   0.6s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, n_estimators=75, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.5s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.5s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.5s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.5s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.6s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100,

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, feature_weights=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=Non...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=1,
             param_grid={'colsample_bytree': [0.7, 0.8, 1.0],
                         'learning_rate': [0.05, 0.01, 0.1],
                         'max_depth': [3, 5], 'n_estimators': [100, 125, 75],
                         'subsample': [0.7, 0.9, 1.0]},
             scoring='accuracy', verbose=2)

In [87]:
grid_search.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.05,
 'max_depth': 3,
 'n_estimators': 100,
 'subsample': 0.9}

In [86]:
best_model=grid_search.best_estimator_
best_model.fit(x_train_full,y_train_full,eval_set=[(x_val,y_val)],verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [80]:
train_acc=accuracy_score(y_train_full,best_model.predict(x_train_full))

In [89]:
train_acc

0.8350071736011477

In [85]:
test_acc=accuracy_score(y_test,best_model.predict(x_test))
test_acc

0.5779816513761468

In [90]:
import pickle

In [111]:
with open('model.pkl','wb') as f:
    pickle.dump(model,f)

In [115]:
with open('model.pkl','rb') as f:
    mp=pickle.load(f)
    

In [114]:
mp.predict([[24,4,9,5,0,164.0,20.0,1,0]])

array([9], dtype=int64)

In [125]:
import joblib


In [126]:
joblib.dump(model,'model_joblib')

['model_joblib']

In [128]:
import pickle
with open('model.pkl','wb')as f:
    pickle.dump(model,f)